# Demonstration Notebook
## Improvise melody based on learned musical style

By Ben Walsh \
For Liloquy

&copy; 2021 Ben Walsh <ben@liloquy.io>

## Contents

1. [Import Libraries](#lib_import)
1. [Define Model](#model_define)
1. [Test Model](#model_test)

## <a id = "lib_import"></a>1. Import Libraries

In [32]:
import numpy as np

from pomegranate import State, HiddenMarkovModel, DiscreteDistribution

## <a id = "model_define"></a>2. Define Model

### Define (placeholder) initial note distributions for two genres

In [33]:
note_distro = {}
note_distro['major'] = {
    'C4': 0.3,
    'D4': 0.1,
    'E4': 0.2,
    'F4': 0.05,
    'G4': 0.2,
    'A4': 0.1,
    'B4': 0.05
}

note_distro['minor'] = {
    'C4': 0.2,
    'D4': 0.1,
    'E4': 0.2,
    'F4': 0.05,
    'G4': 0.1,
    'A4': 0.3,
    'B4': 0.05
}

In [34]:
note_distro

{'major': {'C4': 0.3,
  'D4': 0.1,
  'E4': 0.2,
  'F4': 0.05,
  'G4': 0.2,
  'A4': 0.1,
  'B4': 0.05},
 'minor': {'C4': 0.2,
  'D4': 0.1,
  'E4': 0.2,
  'F4': 0.05,
  'G4': 0.1,
  'A4': 0.3,
  'B4': 0.05}}

In [37]:
genre_distro_start = {}
genre_distro_start = {
    'major': 0.5,
    'minor': 0.5
}

genre_distro_end = {}
genre_distro_end = {
    'major': 0.5,
    'minor': 0.5
}

genre_distro_trans = {
    ('major', 'major'): 0.9,
    ('major', 'minor'): 0.1,
    ('minor', 'major'): 0.2,
    ('minor', 'minor'): 0.8,
}

### Define Hidden Markov Model using Pomegranate

In [42]:
# Initialize model
hmm_model = HiddenMarkovModel(name="hmm-note-transitions")

# Initialize the states in the HMM
states = {}

# Add initial distributions
for genre in note_distro.keys():
    
    # Ensure normalization
    sum_values = sum(note_distro[genre].values())
    for k, v in note_distro[genre].items():
        note_distro[genre].update({k:v/sum_values})
    
    # Define HMM state for genre
    states[genre] = State(DiscreteDistribution(note_distro[genre]), name=genre)
    
    # Add state to hmm_model
    hmm_model.add_states(states[genre])

# Add starting genre distributions
for genre in note_distro.keys():
    hmm_model.add_transition(hmm_model.start, states[genre], genre_distro_start[genre])

# Add ending note distributions
for genre in note_distro.keys():
    hmm_model.add_transition(states[genre], hmm_model.end, genre_distro_end[genre])

for (genre1, genre2) in genre_distro_trans.keys():
    hmm_model.add_transition(states[genre1], states[genre2], genre_distro_trans[(genre1, genre2)])

hmm_model.bake()

## <a id = "model_test"></a>3. Test Model 

### Predict genre on input melodies

In [50]:
major_seq = ('C4', 'E4', 'G4', 'C4', 'E4', 'G4')
[hmm_model.states[genre].name for genre in hmm_model.predict(major_seq)]

['major', 'major', 'major', 'major', 'major', 'major']

In [53]:
minor_seq = ('A4', 'C4', 'E4', 'A4', 'C4', 'E4', 'A4')
[hmm_model.states[genre].name for genre in hmm_model.predict(minor_seq)]

['minor', 'minor', 'minor', 'minor', 'minor', 'minor', 'minor']